In [6]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.retrievers import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain.chains import RetrievalQA

pdfLoad = PyPDFLoader('med_data_all.pdf')
documents = pdfLoad.load()

txt_split = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
docs = txt_split.split_documents(documents)
print(len(docs))

embedding = OpenAIEmbeddings(model='text-embedding-ada-002')
vdb = FAISS.from_documents(docs, embedding)
base_retriever = vdb.as_retriever(search_kwargs={"k": 10})

reranker = CohereRerank(
    model="rerank-multilingual-v3.0",  
    top_n=5  # 최종 문서 5개만 선택 (원하는 수로 조정 가능)
)

retriever = ContextualCompressionRetriever(
    base_compressor=reranker,
    base_retriever=base_retriever,
)

gpt = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)

qa = RetrievalQA.from_chain_type(
    llm=gpt,
    chain_type='stuff',
    retriever=retriever,
)

789


In [ ]:
query = "고혈압 치료 방법"
result = qa.invoke({'query': query})
print(result)
print(result['result'])

In [8]:
with open('C:\workspace\silverNews\saved_articles\Hypertension_1_20250521.txt', 'r', encoding='utf-8') as f:
    article = f.read()

# print(article)

# 유사도 확인을 위한 쿼리 예시
query = f"너는 30년 경력의 의사야. 기사를 요약해줘. 기사: {article}"

# result = qa.invoke({'query': query})
# print(result)

# 기본 검색 결과
# base_results = base_retriever.get_relevant_documents(query)
# print("기본 검색 결과 수:", len(base_results))

# 리랭킹된 결과
reranked_results = retriever.get_relevant_documents(query)
print("리랭킹된 결과 수:", len(reranked_results))

# 결과 비교
# print("\n=== 기본 검색 결과 ===")
# for i, doc in enumerate(base_results):
#     print(f"\n문서 {i+1}:")
#     print(f"내용: {doc.page_content[:200]}...")
#     print(f"메타데이터: {doc.metadata}")

# print("\n=== 리랭킹된 결과 ===")
# for i, doc in enumerate(reranked_results):
#     print(f"\n문서 {i+1}:")
#     print(f"내용: {doc.page_content[:200]}...")
#     print(f"메타데이터: {doc.metadata}")

# 유사도 점수 확인
print("\n=== 유사도 점수 ===")
for i, doc in enumerate(reranked_results):
    print(doc.page_content)
    print(doc.metadata)
    print('-'*100)
    if hasattr(doc, 'score'):
        print(f"문서 {i+1} 유사도 점수: {doc.score}")

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\김태원\AppData\Local\Temp\ipykernel_20252\3751528276.py:1: SyntaxWarning: invalid escape sequence '\w'
  with open('C:\workspace\silverNews\saved_articles\Hypertension_1_20250521.txt', 'r', encoding='utf-8') as f:


리랭킹된 결과 수: 5

=== 유사도 점수 ===
problems such as stroke, heart failure, heart attack, and kidney disease. What lifestyle 
changes can help lower high blood pressure? Healthy lifestyle changes can help reduce 
high blood pressure, losing weight, being physically active, managing stress, reducing 
sodium in your diet, avoiding alcohol, tobacco, and illegal drugs, getting enough sleep. 
What if lifestyle changes alone cannot lower blood pressure? Sometimes lifestyle changes
{'producer': 'Microsoft® Word 2019', 'creator': 'Microsoft® Word 2019', 'creationdate': '2025-05-14T15:44:39+09:00', 'author': 'Nitrosoft', 'moddate': '2025-05-14T15:44:39+09:00', 'source': 'med_data_all.pdf', 'total_pages': 126, 'page': 97, 'page_label': '98', 'relevance_score': 0.008412978}
----------------------------------------------------------------------------------------------------
Other problems include heartburn, cancer, and eosinophilic esophagitis. Doctors may use 
various tests to make a diagnosis. These in

OpenAI의 text-embedding-ada-002 모델 특성
-----------------------------------------------------
text-embedding-ada-002는 단순한 영어 전용이 아니라,
한국어, 영어, 일본어, 중국어 등 다양한 언어를
"공통된 의미 공간(shared semantic space)" 안에서 임베딩한다


"The Adventures of Tom Sawyer"라는 영어 텍스트를 임베딩하면,
그것이 가진 의미(semantics) 가 벡터로 표현됨.
그리고 한글 질문 "줄거리 요약해줘"도 벡터로 임베딩할 때
그 의미가 비슷한 벡터로 변환됨.

결국
"의미가 비슷하면 언어가 달라도 벡터가 가까워진다" 는 것.
